In [1]:
import data_handling
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch
import torchvision
import numpy as np
import encdec as ed

In [2]:
#torch.set_default_dtype(torch.float16)
DATASET_DIRECTORY = "./CelebDataProcessed"
ANNOTATIONS_DIRECTORY = "./annotations.csv"
NAME = ""
BATCH_SIZE = 128
TRANSFORM = torchvision.transforms.Compose([
torchvision.transforms.ToPILImage(),
torchvision.transforms.ToTensor(),
])



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

pubfig = data_handling.PublicFigureDataset(ANNOTATIONS_DIRECTORY, DATASET_DIRECTORY, NAME, transform=TRANSFORM)

train_size = int(0.8 * len(pubfig))
test_size = len(pubfig) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(pubfig, [train_size, test_size])

train_dl = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

: 

: 

In [ ]:
index = 0
print(type(pubfig[index][0]))
print("Image Shape: " + str(pubfig[index][0].size())) # Image
print("Images are of: " + str(pubfig[index][0].dtype))
print((pubfig[index][1])) # Label

<class 'torch.Tensor'>
Image Shape: torch.Size([3, 256, 256])
Images are of: torch.float32
Donald Trump


In [ ]:
### Test 1: Encoder - Decoder model should be able to successfully pass data through to a reconstruction

### Test 2: Training should work

### Test 3: Saving and loading weights

### Test 4: Loaded model should output relevant reconstructions

In [ ]:
### Encoder Hyperparameters
INPUT_DIM = 256
LAYER_COUNT = 6
LATENT_SIZE = 1024

encoder = ed.Encoder(LAYER_COUNT, LATENT_SIZE, INPUT_DIM)
print(encoder)

Encoder(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Dropout2d(p=0.5, inplace=False)
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Dropout2d(p=0.5, inplace=False)
    )
    (2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Dropout2d(p=0.5, inplace=False)
    )
    (3): Sequential(
      (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(5

In [ ]:
import helper as h
from numpy.random import randint
index = randint(len(pubfig))
image , _= h.getImage(index, pubfig)

encoder.eval()
output = encoder(image.unsqueeze(0)) # Images need to be unsqueezed since we use batch norm. The model expects batch size as a part of the image.
print(output.size())

torch.Size([1, 1024])


In [ ]:
decoder = ed.Decoder(LAYER_COUNT, LATENT_SIZE, INPUT_DIM, encoder.getOut())
print(decoder)

Decoder(
  (decoder_lin): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=2048, out_features=8192, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
  (unflatten): Unflatten(dim=1, unflattened_size=(512, 4, 4))
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): ConvTranspose2d(2048, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Dropout2d(p=0.5, inplace=False)
    )
    (2): Sequential(
      (0): ConvTranspose2d(1024, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope

In [ ]:
decoder.eval()
output = decoder(output)
print(output.size())

torch.Size([1, 3, 256, 256])


C:\Users\Kam Nanthanolath\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
### Training Parameters
import models


model = models.AutoEncoder(LAYER_COUNT, LATENT_SIZE, INPUT_DIM)
#h.loadWeights(model, "./5Epochs1024Lat")
print(model)
EPOCHS = 50000
loss_fn = torch.nn.MSELoss()
lr= 0.001
optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-05)


AutoEncoder(
  (encoder): Encoder(
    (model): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
        (3): Dropout2d(p=0.5, inplace=False)
      )
      (1): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
        (3): Dropout2d(p=0.5, inplace=False)
      )
      (2): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
        (3): Dropout2d(p=0.5, inplace=False)
      )
      (3): Sequential(
        (0): Conv2d(256, 512, kernel_

In [ ]:


h.train_epoch(model,device, train_dl,loss_fn,optim, test_dl, pubfig, epochs=EPOCHS, video=True)

Epoch: 1 | Iter:    0 | Time: 00:00:07 | Train Loss: 0.3362 | Average Loss: 0.3362 

 EPOCH 1/50000 	 Train loss 0.33620136976242065 	 Test loss 0.04880928620696068
Epoch: 2 | Iter:    0 | Time: 00:00:16 | Train Loss: 0.3411 | Average Loss: 0.3387 

 EPOCH 2/50000 	 Train loss 0.33867010474205017 	 Test loss 0.04914069548249245
Epoch: 3 | Iter:    0 | Time: 00:00:25 | Train Loss: 0.3011 | Average Loss: 0.3262 

 EPOCH 3/50000 	 Train loss 0.32615888118743896 	 Test loss 0.04961671307682991
Epoch: 4 | Iter:    0 | Time: 00:00:34 | Train Loss: 0.2905 | Average Loss: 0.3172 

 EPOCH 4/50000 	 Train loss 0.31723684072494507 	 Test loss 0.049784015864133835
Epoch: 5 | Iter:    0 | Time: 00:00:43 | Train Loss: 0.2842 | Average Loss: 0.3106 

 EPOCH 5/50000 	 Train loss 0.31062257289886475 	 Test loss 0.04996086657047272
Epoch: 6 | Iter:    0 | Time: 00:00:53 | Train Loss: 0.2865 | Average Loss: 0.3066 

 EPOCH 6/50000 	 Train loss 0.30659669637680054 	 Test loss 0.05016924440860748
Epoch: 7 

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 149339648 bytes.

In [ ]:
model.eval()
h.saveWeights(model, "./Trump.pth")

: 

In [ ]:
image, _ = h.getImage(12, pubfig)
image = image.unsqueeze(0)

In [ ]:
model.eval()
z = model.encode(image)
output = model.decode(z)

In [ ]:
output.size()
print(output.squeeze().size())
h.showImage(output.squeeze())
h.showImage(image.squeeze())

torch.Size([3, 256, 256])


In [ ]:
# h.saveWeights(model, "./Trump")

In [ ]:
from numpy.random import randint

for i in range(10):
    index = randint(len(pubfig))
    image, label = h.getImage(index)
    z = model.encode(image.unsqueeze(0))
    output = model.decode(z)
    h.save_image(image, "./Outputs/5Epochs1024Lat/input{}.png".format(label))
    h.save_image(output, "./Outputs/5Epochs1024Lat/output{}.png".format(label))

C:\Users\Kam Nanthanolath\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
### Load weights
model2 = models.AutoEncoder(LAYER_COUNT, LATENT_SIZE, INPUT_DIM)
h.loadWeights(model2, "./5Epochs1024Lat")

model2.eval()
z = model2.encode(image.unsqueeze(0))
output = model2.decode(z)
h.showImage(output.squeeze())